In [ ]:
import os
import math
import chess

import pandas as pd
import numpy as np

# disables warnings from tensorfeed
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.callbacks import ModelCheckpoint # type: ignore
from tensorflow.keras import layers, models # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from sklearn.model_selection import train_test_split

In [116]:
data_paths = ["../data/train-00000-of-00017.parquet"]
piece_map = {"P": 0, "N": 1, "B": 2, "R": 3, "Q": 4, "K": 5,
             "p": 6, "n": 7, "b": 8, "r": 9, "q": 10, "k": 11}

def board_to_tensor(board: chess.Board):
    """
    Converts a chess.Board object into a (8, 8, 19) tensor input
    """
    tensor = np.zeros((8, 8, 19), dtype=np.float32)
    for square, piece in board.piece_map().items():
        rank, file = divmod(square, 8)
        tensor[rank, file, piece_map[piece.symbol()]] = 1.0
    
    if board.turn == chess.WHITE: tensor[:, :, 12] = 1.0
    if board.has_kingside_castling_rights(chess.WHITE): tensor[:, :, 13] = 1.0
    if board.has_queenside_castling_rights(chess.WHITE): tensor[:, :, 14] = 1.0
    if board.has_kingside_castling_rights(chess.BLACK): tensor[:, :, 15] = 1.0
    if board.has_queenside_castling_rights(chess.BLACK): tensor[:, :, 16] = 1.0
    
    if board.ep_square:
        r, f = divmod(board.ep_square, 8)
        tensor[r, f, 17] = 1.0
    tensor[:, :, 18] = board.halfmove_clock / 100.0
    
    return tensor

def normalise_evaluation(cp, mate):
    """
    Normalises chess evaluations to a [-1, 1] range.
    """
    if not np.isnan(mate):
        # If there's a mate, it's either 1.0 (white wins) or -1.0 (black wins)
        return 1.0 if mate > 0 else -1.0
    
    # Scale centipawns. Using a tanh-style curve: 
    return math.tanh(cp / 5000)

def load_parquet(path: str, frac: int = 1) -> pd.DataFrame:
    df = pd.read_parquet(path)
    return df.head(math.floor(df.shape[0] / frac))

In [117]:
FRACTION = 4
df = load_parquet(data_paths[0], FRACTION)

In [118]:
class TensorGenerator:
    def __init__(self, df, batch_size=64):
        self.df = df
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.df))

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))

    def get_batch(self, batch_idx):
        start = batch_idx * self.batch_size
        end = (batch_idx + 1) * self.batch_size
        
        batch_df = self.df.iloc[start:end]
        
        X = np.zeros((self.batch_size, 8, 8, 19), dtype=np.float32)
        y = np.zeros((self.batch_size, 1), dtype=np.float32)

        for i, (_, row) in enumerate(batch_df.iterrows()):
            board = chess.Board(row["fen"])
            X[i] = board_to_tensor(board)
            y[i] = normalise_evaluation(row["cp"], row["mate"])
            
        return X, y

    def feed(self):
        """A generator function for model.fit()"""
        while True:
            np.random.shuffle(self.indexes)
            for i in range(self.__len__()):
                yield self.get_batch(i)

In [119]:
train_df, val_df = train_test_split(df, test_size=0.1)

In [120]:
train_gen = TensorGenerator(train_df, batch_size=128)
val_gen = TensorGenerator(val_df, batch_size=128)

In [121]:
def build_model():
    # Input shape is (8, 8, 19)
    inputs = layers.Input(shape=(8, 8, 19))

    # Convolutional layers to find spatial patterns (pinning, forks, pawn chains)
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    
    x = layers.Dense(512, activation="relu")(x)
    x = layers.Dropout(0.4)(x) 
    
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(64, activation="relu")(x)

    output = layers.Dense(1, activation="tanh")(x)

    optimiser = Adam(learning_rate=0.0005) 
    model = models.Model(inputs=inputs, outputs=output)
    model.compile(optimizer=optimiser, loss="huber", metrics=["mae"])
    return model

model = build_model()
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 8, 8, 19)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 8, 8, 64)       │        11,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 8, 8, 64)       │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,553,281 (9.74 MB)

 Trainable params: 2,552,513 (9.74 MB)

 Non-trainable params: 768 (3.00 KB)

In [ ]:
model_path = "../models/tensorfish.keras"
checkpoint_callback = ModelCheckpoint(
    filepath=model_path,
    save_best_only=True,    # Only overwrite if the model is better than the previous version
    monitor="val_loss",     # Look at validation loss
    mode="min",             # Lower loss is better
    verbose=1
)

In [124]:
EPOCHS=5
model.fit(
    train_gen.feed(),
    epochs=EPOCHS,
    steps_per_epoch=len(train_gen),
    validation_data=val_gen.feed(),
    validation_steps=len(val_gen),
    callbacks=[checkpoint_callback]
)

Epoch 1/5
87354/87354 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0197 - mae: 0.0821
Epoch 1: val_loss improved from None to 0.01519, saving model to models/tensorfish.keras

Epoch 1: finished saving model to models/tensorfish.keras
87354/87354 ━━━━━━━━━━━━━━━━━━━━ 2428s 28ms/step - loss: 0.0176 - mae: 0.0761 - val_loss: 0.0152 - val_mae: 0.0676
Epoch 2/5
87354/87354 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0147 - mae: 0.0675
Epoch 2: val_loss improved from 0.01519 to 0.01354, saving model to models/tensorfish.keras

Epoch 2: finished saving model to models/tensorfish.keras
87354/87354 ━━━━━━━━━━━━━━━━━━━━ 2362s 27ms/step - loss: 0.0142 - mae: 0.0658 - val_loss: 0.0135 - val_mae: 0.0624
Epoch 3/5
87353/87354 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0132 - mae: 0.0625
Epoch 3: val_loss improved from 0.01354 to 0.01276, saving model to models/tensorfish.keras

Epoch 3: finished saving model to models/tensorfish.keras
87354/87354 ━━━━━━━━━━━━━━━━━━━━ 2348s 27ms/step - loss: 0.0129 -

In [126]:
model.save("../models/tensorfish.keras")